# Лекция 05. Конструирование признаков (Feature Engineering) и отбор признаков (Feature Selection)
В этом курсе, мы уже видели несколько ключевых алгоритмов машинного обучения. Однако, прежде чем перейти к более сложным алгоритмам, мы хотели бы сделать небольшое отступление и поговорить о подготовке данных. Известная концепция "garbage in — garbage out(мусор на входе-мусор на выходе)" применима на 100% к любой задаче в машинном обучении. Любой опытный специалист может вспомнить множество случаев, когда простая модель, обученная на высококачественных данных, оказывалась лучше, чем сложный ансамбль из нескольких моделей, построенный на данных, которые были недостаточно чистыми.

Для начала я хотел рассмотреть три похожие, но разные задачи:
* **feature extraction** и **feature engineering**: преобразование необработанных данных в объекты, пригодные для моделирования;
* **feature transformation**: преобразование данных для повышения точности алгоритма;
* **feature selection**: удаление ненужных признаков.

Эта статья почти не будет содержать математику, но будет довольно много кода. В некоторых примерах будет использоваться набор данных от компании Renthop, который используется в [Two Sigma Connect: Rental Listing Inquiries Kaggle competition](https://www.kaggle.com/c/two-sigma-connect-rental-listing-inquiries). Файл `train.json` также хранится [здесь](https://drive.google.com/open?id=1_lqydkMrmyNAgG4vU4wVmp6-j7tV0XI8) как `renthop_train.json.gz` (так что, сначала распакуйте его). В этой задаче вам нужно спрогнозировать популярность объявлений об аренде, т. е. разделите листинг объявлений на три класса:`['low', 'medium' , 'high']`. Для оценки решений мы будем использовать показатель log loss (чем меньше, тем лучше). Те, у кого нет учетной записи Kaggle, должны будут зарегистрироваться; вам также нужно будет принять правила конкурса, чтобы загрузить данные. 

#автоматическая предварительная загрузка набора данных, если он еще не установлен.
import os, requests

url = 'https://drive.google.com/uc?export=download&id=1_lqydkMrmyNAgG4vU4wVmp6-j7tV0XI8'
file_name = '../../data/renthop_train.json.gz'

def load_renthop_dataset(url, target, overwrite=False):
    #check if exists already
    if os.path.isfile(target) and not overwrite:
        print("Dataset is already in place")
        return
    
    print("Will download the dataset from", url)
    
    response = requests.get(url)
    open(target, 'wb').write(response.content)

load_renthop_dataset(url, file_name)

In [4]:
import numpy as np 
import pandas as pd

df = pd.read_json("../../data/renthop_train.json.gz", compression='gzip')

## Краткое содержание

1. Feature Extraction (Извлечение признаков)
        1. Тексты
        2. Изображения
        3. Геоданные
        4. Дата и время
        5. Временные ряды, веб и т.д.

2. Feature transformations (Преобразование признаков)
        1. Нормализация и изменение распределения
        2. Взаимодействия
        3. Заполнение пропусков

3. Feature selection (Выбор признаков)
        1. Статистические подходы
        2. Выбор путем построения модели
        3. Перебор

## Feature Extraction

На практике данные редко поступают в виде готовых к использованию матриц. Вот почему каждая задача начинается с извлечения признаков. Иногда, может быть достаточно, прочитать файл csv и конвертировать его в `numpy.array`, но это редкое исключение. Давайте рассмотрим некоторые популярные типы данных, из которых можно извлечь признаки.

### Тексты

Текст-это тип данных, который может поступать в разных форматах; существует множество методов обработки текста, которые не могут поместиться в одной статье. Тем не менее, мы рассмотрим самые популярные из них.

Прежде чем работать с текстом, нужно его токенизировать. Токенизация подразумевает разбиение текста на единицы (следовательно, токены). В самом простом варианте токены - это просто слова. Но разделяя по словам мы может потерять часть смысла - "Санта Барбара" - это один токен, а не два, но слово "рок-н-ролл" должен быть разделен на два токена. Существуют готовые к использованию токенизаторы, учитывающие особенности языка, но они также допускают ошибки, особенно при работе со специфическими текстами (газетами, сленгом, орфографическими ошибками, опечатками).

После токенизации вы нормализуете данные. Для текста, речь идет о stemming'е и/или лемматизации; это похожие процессы, используемые для обработки различных форм слова. Можно прочитать о разнице между ними [здесь](http://nlp.stanford.edu/IR-book/html/htmledition/stemming-and-lemmatization-1.html).

Итак, теперь, когда мы превратили документ в последовательность слов, мы можем представить его в качестве векторов. Самый простой подход называется Bag of Words (мешок слов): мы создаем вектор с длиной словаря, вычисляем количество вхождений каждого слова в тексте и помещаем это количество вхождений в соответствующую позицию в векторе. Описанный процесс проще выглядит в коде:

In [5]:
texts = ['i have a cat', 
         'you have a dog', 
         'you and i have a cat and a dog']

vocabulary = list(enumerate(set([word for sentence 
                                 in texts for word in sentence.split()])))
print('Vocabulary:', vocabulary)

def vectorize(text): 
    vector = np.zeros(len(vocabulary)) 
    for i, word in vocabulary:
        num = 0 
        for w in text: 
            if w == word: 
                num += 1 
        if num: 
            vector[i] = num 
    return vector

print('Vectors:')
for sentence in texts: 
    print(vectorize(sentence.split()))

Vocabulary: [(0, 'dog'), (1, 'cat'), (2, 'and'), (3, 'you'), (4, 'i'), (5, 'have'), (6, 'a')]
Vectors:
[ 0.  1.  0.  0.  1.  1.  1.]
[ 1.  0.  0.  1.  0.  1.  1.]
[ 1.  1.  2.  1.  1.  1.  2.]


Вот иллюстрация этого процесса:

<img src='../../img/bag_of_words.png' width=50%>

Это крайне наивная реализация. На практике необходимо учитывать стоп-слова, максимальную длину словаря, более эффективные структуры данных (обычно текстовые данные преобразуются в разреженный вектор) и т.д.

При использовании алгоритмов типа Bag of Words мы теряем порядок слов в тексте, что означает, что тексты "у меня нет коров" и "нет, у меня есть коровы" будут выглядеть идентичными после векторизации, когда на самом деле они имеют противоположное значение. Чтобы избежать этой проблемы, мы можем вернуться к нашему шагу токенизации и использовать вместо этого N-граммы (последовательность *из N последовательных токенов).

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer(ngram_range=(1,1))
vect.fit_transform(['no i have cows', 'i have no cows']).toarray()

array([[1, 1, 1],
       [1, 1, 1]])

In [7]:
vect.vocabulary_ 

{'no': 2, 'have': 1, 'cows': 0}

In [8]:
vect = CountVectorizer(ngram_range=(1, 2))
vect.fit_transform(['no i have cows', 'i have no cows']).toarray()

array([[1, 1, 1, 0, 1, 0, 1],
       [1, 1, 0, 1, 1, 1, 0]])

In [9]:
vect.vocabulary_

{'no': 4,
 'have': 1,
 'cows': 0,
 'no have': 6,
 'have cows': 2,
 'have no': 3,
 'no cows': 5}

Также обратите внимание, что не нужно использовать только слова. В некоторых случаях можно сгенерировать N-граммы символов. Этот подход мог бы учитывать сходство связанных слов или обрабатывать опечатки.

In [10]:
from scipy.spatial.distance import euclidean
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer(ngram_range=(3, 3), analyzer='char_wb')

n1, n2, n3, n4 = vect.fit_transform(['andersen', 'petersen', 'petrov', 'smith']).toarray()

euclidean(n1, n2), euclidean(n2, n3), euclidean(n3, n4)

(2.8284271247461903, 3.1622776601683795, 3.3166247903553998)

Развитие идеи Bag of Words: слова, которые редко встречаются в корпусе (во всех рассматриваемых документах этого набора данных), но присутствуют в этом конкретном документе, могут оказаться более важными. Тогда имеет смысл повысить вес более узкотематическим словам, чтобы отделить их от общетематических. Этот подход называется TF-IDF (term frequency-inverse document frequency), его уже не напишешь в несколько строк кода, потому желающие могут ознакомиться с деталями во внешних источниках вроде [wiki](https://en.wikipedia.org/wiki/Tf%E2%80%93idf). Вариант по умолчанию выглядит так:

$$ \large idf(t,D) = \log\frac{\mid D\mid}{df(d,t)+1} $$

$$ \large tfidf(t,d,D) = tf(t,d) \times idf(t,D) $$

Аналоги Bag of words могут встречаться и за пределами текстовых задач: например, bag of sites в соревнавании [Catch Me If You Can](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking), [bag of apps](https://www.kaggle.com/xiaoml/talkingdata-mobile-user-demographics/bag-of-app-id-python-2-27392), [bag of events](http://www.interdigital.com/download/58540a46e3b9659c9f000372), etc.

![image](../../img/bag_of_words.png)

Используя эти алгоритмы, можно получить рабочее решение для простой задачи, которое может служить в качестве базы (baseline). Однако для тех, кто не любит классику, существуют новые подходы. Самый популярный метод новой волны - [Word2Vec](https://arxiv.org/pdf/1310.4546.pdf), но есть и альтернативы ([GloVe](https://nlp.stanford.edu/pubs/glove.pdf), [Fasttext](https://arxiv.org/abs/1607.01759), etc.).

Word2Vec является частным случаем алгоритмов векторного представления (word embedding). Используя Word2Vec и подобные модели, мы можем не только векторизовать слова в многомерном пространстве (обычно несколько сотен измерений), но и сравнить их семантическое сходство. Это классический пример операций, которые могут быть выполнены на векторизованных представлениях: король-мужчина + женщина = королева (king - man + woman = queen).

![image](https://cdn-images-1.medium.com/max/800/1*K5X4N-MJKt8FGFtrTHwidg.gif)

Стоит отметить, что эта модель не обладает пониманием слов, а просто пытается расположить векторы таким образом, чтобы слова, используемые в общем контексте, были близки друг к другу. Если это не будет учтено, появится много забавных курьезов.

Такие модели должны быть обучены на очень больших наборах данных, чтобы векторные координаты могли обхватить всю семантику. Предварительно подготовленную модель для ваших собственных задач можно загрузить [здесь](https://github.com/3Top/word2vec-api#where-to-get-a-pretrained-models).

Аналогичные методы применяются и в других областях, таких как биоинформатика. Неожиданным применением является [food2vec](https://jaan.io/food2vec-augmented-cooking-machine-intelligence/). Вы, вероятно, можете придумать несколько других свежих идей; концепция достаточно универсальна.

### Изображения

Работать с изображениями проще и сложнее одновременно. Это проще, потому что можно просто использовать одну из популярных предварительно обученных сетей без особых размышлений, но сложнее, потому что, если вам нужно копаться в деталях, вы можете в конечном итоге пойти очень глубоко. Давайте начнем с самого начала.

В то время, когда графические процессоры были слабее и "Ренессанс нейронных сетей" еще не случился, генерация признаков из изображений была особенно сложной областью. Приходилось работать на низком уровне, определяя углы, границы областей, статистику распределения цветов и так далее. Опытные специалисты в области компьютерного зрения могли бы провести много параллелей между старыми подходами и нейронными сетями; в частности, сверточные слои в современных сетях похожи на [каскады Хаара](https://en.wikipedia.org/wiki/Haar-like_feature). Если вы захотите прочитать больше, вот несколько ссылок на некоторые интересные библиотеки: [skimage](http://scikit-image.org/docs/stable/api/skimage.feature.html) и [SimpleCV](http://simplecv.readthedocs.io/en/latest/SimpleCV.Features.html).

Часто для решения задач, связанных с изображениями, используется сверточная нейронная сеть. Вам не придется придумывать архитектуру и обучать сеть с нуля. Вместо этого загрузите предварительно подготовленную state of the art сеть с весами из открытых источников. Специалисты по обработке данных часто делают так называемую тонкую настройку, чтобы адаптировать эти сети к своим потребностям, "отсоединяя" последние полностью подключенные слои сети и добавляя новые слои, выбранные для конкретной задачи, а затем обучая сеть на новых данных. Если ваша задача состоит в том, чтобы просто векторизовать изображение (например, использовать какой-то несетевой классификатор), вам нужно только удалить последние слои и использовать выходные данные из предыдущих слоев:

In [10]:
# doesn't work with Python 3.7 
# # Install Keras and tensorflow (https://keras.io/)
# from keras.applications.resnet50 import ResNet50, preprocess_input
# from keras.preprocessing import image 
# from scipy.misc import face 
# import numpy as np

# resnet_settings = {'include_top': False, 'weights': 'imagenet'}
# resnet = ResNet50(**resnet_settings)

# # What a cute raccoon!
# img = image.array_to_img(face())
# img

In [11]:
# # In real life, you may need to pay more attention to resizing
# img = img.resize((224, 224))

# x = image.img_to_array(img) 
# x = np.expand_dims(x, axis=0)
# x = preprocess_input(x)

# # Need an extra dimension because model is designed to work with an array
# # of images - i.e. tensor shaped (batch_size, width, height, n_channels)

# features = resnet.predict(x)

<img src='https://cdn-images-1.medium.com/max/800/1*Iw_cKFwLkTVO2SPrOZU2rQ.png' width=60%>

*Вот классификатор, обученный на одном наборе данных и адаптированный для другого, "отсоединием" последнего слоя и добавлением вместо него нового.*

Тем не менее, мы не должны слишком сосредотачиваться на нейросетевых методах. Функции, генерируемые вручную, по-прежнему очень полезны: например, для прогнозирования популярности объявления аренды можно предположить, что светлые квартиры привлекают больше внимания и создают такой признак, как "среднее значение пикселя". Вы можете найти некоторые вдохновляющие примеры в документации [соответствующих библиотек](http://pillow.readthedocs.io/en/3.1.x/reference/ImageStat.html).

Если на картинке ожидается текст, его также можно прочитать и не разворачивая своими руками сложную нейросеть: например, при помощи [pytesseract](https://github.com/madmaze/pytesseract).

```python
import pytesseract
from PIL import Image
import requests
from io import BytesIO

##### Just a random picture from search
img = 'http://ohscurrent.org/wp-content/uploads/2015/09/domus-01-google.jpg'

img = requests.get(img)
img = Image.open(BytesIO(img.content))
text = pytesseract.image_to_string(img)

text

Out: 'Google'
```

Нужно понимать, что `pytesseract` - это решение не для всех случаев.

```python
##### This time we take a picture from Renthop
img = requests.get('https://photos.renthop.com/2/8393298_6acaf11f030217d05f3a5604b9a2f70f.jpg')
img = Image.open(BytesIO(img.content))
pytesseract.image_to_string(img)

Out: 'Cunveztible to 4}»'
```

Другой случай, когда нейронные сети не могут помочь, - это извлечение признаков из метаинформации. У изображений файл EXIF хранит много полезной метаинформации: производитель и модель камеры, разрешение, использование вспышки, географические координаты съемки, программное обеспечение, используемое для обработки изображения и многое другое.

### Геоданные

Географические данные не так часто встречаются в задачах, но все же полезно овладеть основными приемами работы с ними, тем более что в этой области существует довольно много готовых решений.

Геопространственные данные часто представляются в виде адресов или координат (широта, долгота). В зависимости от поставленной задачи могут потребоваться две взаимно-обратные операции: геокодирование (восстановление точки из адреса) и обратное геокодирование (восстановление адреса из точки). Обе операции доступны на практике через внешние API из Google Maps или OpenStreetMap. Различные геокодеры имеют свои особенности, и качество варьируется от региона к региону. К счастью, существуют универсальные библиотеки, такие как [geopy](https://github.com/geopy/geopy), которые действуют как оболочки для этих внешних служб.

Если у вас много данных, вы быстро достигнете пределов внешнего API. Кроме того, это не всегда самый быстрый способ получить информацию через HTTP. Следовательно, необходимо рассмотреть возможность использования локальной версии проекта OpenStreetMap.

Если у вас есть небольшой объем данных, достаточно времени, и нет желания извлекать причудливые признаки, вы можете использовать `reverse_geocoder` вместо OpenStreetMap:

```python
import reverse_geocoder as revgc

revgc.search((df.latitude, df.longitude))
Loading formatted geocoded file... 

Out: [OrderedDict([('lat', '40.74482'), 
                   ('lon', '-73.94875'), 
                   ('name', 'Long Island City'), 
                   ('admin1', 'New York'), 
                   ('admin2', 'Queens County'), 
                   ('cc', 'US')])]
```

При работе с геокодированием не следует забывать, что адреса могут содержать опечатки, что говорит о необходимости очистки данных. Координаты содержат меньше опечаток, но их положение может быть неправильным из-за шума GPS или плохой точности в таких местах, как туннели, районы в центре города и т.д. Если источником данных является мобильное устройство, геолокация может определяться не по GPS, а сетями WiFi в этом районе, что приводит к дырам в пространстве и телепортации. Во время путешествия по Манхэттену, вдруг может быть точка Wi-Fi из Чикаго.

> Отслеживание местоположения WiFi основано на комбинации SSID и MAC-адресов, которые могут соответствовать различным точкам, например, федеральный провайдер стандартизирует прошивку маршрутизаторов до MAC-адреса и размещает их в разных городах. Даже переезд компании в другой офис с ее маршрутизаторами может вызвать проблемы.

Точка, как правило, находится среди инфраструктуры. Здесь вы действительно можете раскрыть свое воображение и придумать параметры, основанные на вашем жизненном опыте и знании предметной области: близость точки к метро, количество этажей в здании, расстояние до ближайшего магазина, количество банкоматов вокруг и т.д. Для любой задачи вы можете легко придумать десятки параметров и извлечь их из различных внешних источников. Для проблем вне городской среды, вы можете рассмотреть признаки из более конкретных источников, например, высота над уровнем моря.

Если две или более точек связаны между собой, возможно, стоит извлечь объекты из маршрута между ними. В этом случае будут полезны расстояния (стоит смотреть и на great circle distance, и на "честное" расстояние, посчитанное по дорожному графу), количество поворотов с соотношением левых и правых поворотов, количество светофоров, перекрестков и мостов. В одной из моих собственных задач я создал функцию под названием "сложность дороги", которая вычисляла графически рассчитанное расстояние, разделенное на GCD.

### Дата и время

Можно подумать, что дата и время стандартизированы из-за их распространенности, но тем не менее, некоторые подводные камни остаются.

Начнем с дня недели, который легко превратить в 7 фиктивных(dummy) переменных, используя однократное(one-hot) кодирование. Кроме того, мы также создадим отдельную двоичную функцию для выходных под названием `is_weekend`.

```python
df['dow'] = df['created'].apply(lambda x: x.date().weekday())
df['is_weekend'] = df['created'].apply(lambda x: 1 if x.date().weekday() in (5, 6) else 0)
```

Для некоторых задач могут потребоваться дополнительные функции календаря. Например, снятие наличных может быть привязано к платежному дню; покупка карты метро - к началу месяца. В целом, при работе с данными временных рядов неплохо иметь календарь с праздничными днями, ненормальными погодными условиями и другими важными событиями.

> Вопрос: Что общего у китайского Нового года, Нью-Йоркского марафона и инаугурации Трампа?

> Ответ: все они должны быть занесены в календарь потенциальных аномалий.

Работать с часом (минутой, днем месяца ...) не так просто, как кажется. Если вы используете час как реальную переменную, мы немного противоречим природе данных: `0<23`, в то время как`0:00:00 02.01> 01.01 23:00:00`. Для некоторых проблем это может иметь решающее значение. В то же время, если вы закодируете их как категориальные переменные, вы породите большое количество признаков и потеряете информацию о близости-разница между 22 и 23 будет такой же, как разница между 22 и 7.

Существуют также некоторые более эзотерические подходы к таким данным, как проецирование времени на круг и использование двух координат.

In [12]:
def make_harmonic_features(value, period=24):
    value *= 2 * np.pi / period 
    return np.cos(value), np.sin(value)

Это преобразование сохраняет расстояние между точками, что важно для алгоритмов, оценивающих расстояние (kNN, SVM, k-means ...).

In [13]:
from scipy.spatial import distance
euclidean(make_harmonic_features(23), make_harmonic_features(1)) 

0.5176380902050424

In [14]:
euclidean(make_harmonic_features(9), make_harmonic_features(11)) 

0.5176380902050414

In [15]:
euclidean(make_harmonic_features(9), make_harmonic_features(21))

2.0

Однако разница между такими методами кодирования сводится к третьему десятичному знаку в метрике.

### Временные ряды, web, и т.д.

Что касается временных рядов — мы не будем вдаваться в детали здесь (в основном из - за моего личного отсутствия опыта), но я укажу вам на [полезную библиотеку, которая автоматически генерирует функции для временных рядов](https://github.com/blue-yonder/tsfresh).

Если вы работаете с веб-данными, то обычно у вас есть информация о User Agent пользователя. Это огромное количество информации. Во-первых, нужно извлечь из него операционную систему. Во-вторых, сделайть пизнак `is_mobile`. В-третьих, посмотреть на браузер.

In [2]:
# Install pyyaml ua-parser user-agents
import user_agents

ua = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/56.0.2924.76 Chrome/56.0.2924.76 Safari/537.36'
ua = user_agents.parse(ua)

print('Is a bot? ', ua.is_bot)
print('Is mobile? ', ua.is_mobile)
print('Is PC? ',ua.is_pc)
print('OS Family: ',ua.os.family)
print('OS Version: ',ua.os.version)
print('Browser Family: ',ua.browser.family)
print('Browser Version: ',ua.browser.version)

Is a bot?  False
Is mobile?  False
Is PC?  True
OS Family:  Ubuntu
OS Version:  ()
Browser Family:  Chromium
Browser Version:  (56, 0, 2924)


> Как и в других областях, вы можете придумать свои собственные признаки, основанные на интуиции о природе данных. На момент написания этой статьи Chromium 56 был новым, но через некоторое время только пользователи, которые не перезагружали свой браузер в течение длительного времени, будут иметь эту версию. В этом случае, почему бы не ввести функцию под названием "отставание от последней версии браузера"?

В дополнение к операционной системе и браузеру, вы можете посмотреть на реферер (не всегда доступный), [http_accept_language](https://developer.mozilla.org/en-US/docs/Web/HTTP/Headers/Accept-Language), и другую метаинформацию.

Следующая полезная информация-это IP-адрес, из которого можно извлечь страну и, возможно, город, провайдера и тип подключения (мобильный/стационарный). Вы должны понимать, что существует множество прокси и устаревших баз данных, поэтому эта функция может содержать шум. Гуру сетевого администрирования могут попытаться извлечь даже более сложные пизнаки, такие как предложения по [использованию VPN](https://habrahabr.ru/post/216295/). Кстати, данные с IP-адреса хорошо сочетаются с `http_accept_language`: если пользователь сидит на чилийских прокси, а локаль браузера `ru_RU`, то что-то нечисто и стоит посмотреть в соответствующем столбце таблицы (`is_traveler_or_proxy_user`).

Любая данная область имеет так много особенностей, что это слишком много для человека, чтобы узнать о них полностью. Поэтому я приглашаю всех поделиться своим опытом и обсудить извлечение и генерацию признаков в разделе комментариев.

## Преобразование объектов

### Нормализация и изменение распределения

Монотонное преобразование признаков имеет решающее значение для некоторых алгоритмов и не влияет на другие. Это одна из причин возросшей популярности деревьев решений и всех их производных алгоритмов (случайный лес, градиентный бустинг). Не все могут или хотят возиться с преобразованиями, а эти алгоритмы устойчивы к необычным распределениям.

Есть и чисто инженерные причины: `np.log` - это способ работы с большими числами, которые не вписываются в `np.float64`. Это скорее исключение, чем правило; часто оно вызвано желанием адаптировать набор данных к требованиям алгоритма. Параметрические методы обычно требуют минимального симметричного и унимодального распределения данных, которое не всегда дается в реальных данных. Там могут быть более жесткие требования; напомним [предыдущую лекцию о линейных моделях](https://medium.com/open-machine-learning-course/open-machine-learning-course-topic-4-linear-classification-and-regression-44a41b9b5220).

Однако требования к данным предъявляют не только параметрические методы; [K nearest neighbors](https://medium.com/open-machine-learning-course/open-machine-learning-course-topic-3-classification-decision-trees-and-k-nearest-neighbors-8613c6b6d2cd) будет предсказывать полную бессмыслицу, если объекты не нормализованы, например: когда одно распределение находится в окрестности нуля и не выходит за пределы (-1, 1), в то время как диапазон другого порядка сотен тысяч.

Простой пример: предположим, что задача состоит в том, чтобы спрогнозировать стоимость квартиры по двум переменным — удаленности от центра города и количеству комнат. Количество комнат редко превышает 5, тогда как расстояние от центра города легко может измеряться тысячами метров.

Простейшее преобразование - это стандартное масштабирование (Standard Scaling) (или Z-score normalization):

$$ \large z= \frac{x-\mu}{\sigma} $$

Обратите внимание, что Standard Scaling не делает распределение нормальным в строгом смысле этого слова.

In [ ]:
from sklearn.preprocessing import StandardScaler
from scipy.stats import beta
from scipy.stats import shapiro
import numpy as np

data = beta(1, 10).rvs(1000).reshape(-1, 1)
shapiro(data)

In [ ]:
# Value of the statistic, p-value
shapiro(StandardScaler().fit_transform(data))

# With such p-value we'd have to reject the null hypothesis of normality of the data

Но, в какой-то степени, защищает от выбросов:

In [ ]:
data = np.array([1, 1, 0, -1, 2, 1, 2, 3, -2, 4, 100]).reshape(-1, 1).astype(np.float64)
StandardScaler().fit_transform(data)

In [ ]:
(data - data.mean()) / data.std()

Еще одним довольно популярным вариантом является MinMax Scaling, который переносит все точки в заданный интервал (обычно (0, 1)).

$$ \large X_{norm}=\frac{X-X_{min}}{X_{max}-X_{min}} $$

In [ ]:
from sklearn.preprocessing import MinMaxScaler

MinMaxScaler().fit_transform(data)

In [ ]:
(data - data.min()) / (data.max() - data.min()) 

StandardScaling и MinMax Scaling имеют схожие области применения и часто более или менее взаимозаменяемы. Однако если алгоритм предполагает вычисление расстояний между точками или векторами, то по умолчанию используется StandardScaling. Нри этом Min Max Scaling полезен для визуализации, чтобы перенести признаки на отрезок (0, 255).

Если мы предположим, что некоторые данные не распределены нормально, но описываются [логонормальным распределением](https://en.wikipedia.org/wiki/Log-normal_distribution), его можно легко преобразовать к нормальному распределению:

In [ ]:
from scipy.stats import lognorm

data = lognorm(s=1).rvs(1000)
shapiro(data)

In [ ]:
shapiro(np.log(data))

Логнормальное распределение подходит для описания зарплат, стоимости ценных бумаг, городского населения, количества комментариев к статьям в интернете и т.д. Однако для применения этой процедуры базовое распределение необязательно должно быть логнормальным; вы можете попробовать применить это преобразование к любому распределению с тяжелым правым хвостом. Кроме того, можно попытаться использовать и другие подобные преобразования, формулируя собственные гипотезы о том, как приблизить имеющееся распределение к нормали. Примерами таких преобразований являются [преобразование Бокса-Кокса](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.boxcox.html) (логарифм-это частный случай преобразования Бокса-Кокса) или [преобразование Yeo-Johnson](https://gist.github.com/mesgarpour/f24769cd186e2db853957b10ff6b7a95) (расширяет диапазон применяемости на отрицательные числа). Кроме того, вы также можете попробовать добавить константу к признаку — `np.log (x + const)`.

В приведенных выше примерах мы работали с синтетическими данными и строго проверяли нормальность с помощью теста Шапиро-Уилка. Давайте попробуем взглянуть на некоторые реальные данные и проверить нормальность с помощью менее формального метода - [Q-Q plot](https://en.wikipedia.org/wiki/Q%E2%80%93Q_plot). Для нормального распределения она будет выглядеть как гладкая диагональная линия, а визуальные аномалии должны быть интуитивно понятны.


![image](../../img/qq_lognorm.png)
Q-Q график для логнормального распределения

![image](../../img/qq_log.png)
Q-Q график для того же распределения после логарифмирования

In [ ]:
# Let's draw plots!
import statsmodels.api as sm

# Let's take the price feature from Renthop dataset and filter by hands the most extreme values for clarity

price = df.price[(df.price <= 20000) & (df.price > 500)]
price_log = np.log(price)

# A lot of gestures so that sklearn didn't shower us with warnings
price_mm = MinMaxScaler().fit_transform(price.values.reshape(-1, 1).astype(np.float64)).flatten()
price_z = StandardScaler().fit_transform(price.values.reshape(-1, 1).astype(np.float64)).flatten()

Q-Q график исходного признака

In [ ]:
sm.qqplot(price, loc=price.mean(), scale=price.std())

Q-Q график признака после StandartScaler. Форма не меняется

In [ ]:
sm.qqplot(price_z, loc=price_z.mean(), scale=price_z.std())

Q-Q график признака после MinMaxScaler. Форма не меняется

In [ ]:
sm.qqplot(price_mm, loc=price_mm.mean(), scale=price_mm.std())

Q-Q график признака после логарифмирования. Все стало значительно лучше!

In [ ]:
sm.qqplot(price_log, loc=price_log.mean(), scale=price_log.std())

Давайте посмотрим, могут ли преобразования как-то помочь реальной модели. Здесь нет серебряной пули (Нет универсального метода. Фредерик Брукс(с)).

### Interactions (Взаимодействия)

Если предыдущие преобразования казались скорее математическими, то эта часть больше касается природы данных; ее можно отнести как к преобразованиям объектов, так и к созданию объектов.

Давайте еще раз вернемся к задаче Two Sigma Connect: Rental Listing Inquiries. Среди признаков в этой задачи есть количество комнат и цена. Логика подсказывает, что стоимость одной комнаты более показательна, чем общая стоимость, поэтому мы можем выделить такуй признак.

In [ ]:
rooms = df["bedrooms"].apply(lambda x: max(x, .5))
# Avoid division by zero; .5 is chosen more or less arbitrarily
df["price_per_bedroom"] = df["price"] / rooms

Вы должны ограничить себя в этом процессе. Если существует ограниченное число признаков, можно создать все возможные их взаимодействия, а затем отсеять ненужные, используя методы, описанные в следующем разделе. Кроме того, не все взаимодействия между признаками должны иметь физический смысл; например, полиномиальные объекты (см. [sklearn.preprocessing.PolynomialFeatures](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html)) часто используются в линейных моделях и практически не поддаются интерпретации.

### Filling in the missing values (Заполнение недостающих значений/пропусков)

Не многие алгоритмы могут работать с пропущенными значениями, а реальный мир часто предоставляет данные с пробелами. К счастью, это одна из задач, для решения которой не требуется никакого творчества. Обе ключевые библиотеки python для анализа данных предоставляют простые в использовании решения: [pandas.DataFrame.fillna](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.fillna.html) и [sklearn.preprocessing.Imputer](http://scikit-learn.org/stable/modules/preprocessing.html#imputation).

Эти решения не имеют никакого волшебства, происходящего за кулисами. Подходы к обработке пропущенных значений довольно просты:

* одирование пропущенных значений отдельным пустым значением типа `n/a` (для категориальных переменных);
* используйте наиболее вероятное значение признака (среднее или медиану для числовых переменных, наиболее распространенное значение для категориальных переменных);
* или, наоборот, кодировать с некоторым экстремальным значением (хорошо для моделей дерева принятия решений, так как это позволяет модели сделать разделение между отсутствующими и не отсутствующими значениями);
* для упорядоченных данных (например, временных рядов) возьмите соседнее значение — следующее или предыдущее.

![image](https://cdn-images-1.medium.com/max/800/0*Ps-v8F0fBgmnG36S.)

Простые в использовании библиотечные решения иногда предлагают придерживаться чего-то вроде `df = df.fillna(0)` и не потеть промежутки. Но это не самое лучшее решение: подготовка данных занимает больше времени, чем построение моделей, поэтому бездумное заполнение пробелов может скрыть ошибку в обработке и испортить модель.

## Feature selection (Выбор признаков)

Зачем вообще нужно было выбирать признаки? Кому-то эта идея может показаться нелогичным, но есть по крайней мере две важные причины избавиться от несущественных признаков. Первое понятно каждому инженеру: чем больше данных, тем выше вычислительная сложность. Пока мы работаем с игрушечными наборами данных, размер данных не является проблемой, но для реальных загруженных производственных систем сотни дополнительных признаков будут вполне ощутимы. Вторая причина заключается в том, что некоторые алгоритмы принимают шум (неинформативный признак) в качестве сигнала и переобучаются.

### Статистические подходы

Наиболее очевидным кандидатом на удаление является признак, значение которого остается неизменным, т. е. он вообще не содержит никакой информации. Если мы будем опираться на эту мысль, то вполне разумно будет сказать, что признаки с низкой дисперсией хуже, чем признаки с высокой дисперсией. Таким образом, можно рассматривать отсечание признаков с дисперсией ниже определенного порога.

In [ ]:
from sklearn.feature_selection import VarianceThreshold
from sklearn.datasets import make_classification

x_data_generated, y_data_generated = make_classification()
x_data_generated.shape

In [ ]:
VarianceThreshold(.7).fit_transform(x_data_generated).shape

In [ ]:
VarianceThreshold(.8).fit_transform(x_data_generated).shape

In [ ]:
VarianceThreshold(.9).fit_transform(x_data_generated).shape

Есть и другие способы, которые также [основаны на классической статистике](http://scikit-learn.org/stable/modules/feature_selection.html#univariate-feature-selection).

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

x_data_kbest = SelectKBest(f_classif, k=5).fit_transform(x_data_generated, y_data_generated)
x_data_varth = VarianceThreshold(.9).fit_transform(x_data_generated)

In [ ]:
logit = LogisticRegression(solver='lbfgs', random_state=17)

In [ ]:
cross_val_score(logit, x_data_generated, y_data_generated, 
                scoring='neg_log_loss', cv=5).mean()

In [ ]:
cross_val_score(logit, x_data_kbest, y_data_generated, 
                scoring='neg_log_loss', cv=5).mean()

In [ ]:
cross_val_score(logit, x_data_varth, y_data_generated, 
                scoring='neg_log_loss', cv=5).mean()

Мы видим, что выбранные нами признаки улучшили качество классификатора. Конечно, этот пример является чисто искусственным, однако этот прием стоит использовать для реальных задач.

### Отбор с использованием моделей

Другой подход заключается в использовании некоторой базовой модели для оценки признаков, поскольку модель четко показывает их важность. Обычно используются два типа моделей: некоторые "деревянные" композиции, такие как [Random Forest](https://medium.com/open-machine-learning-course/open-machine-learning-course-topic-5-ensembles-of-algorithms-and-random-forest-8e05246cbba7) или линейная модель с лассо(Lasso) регуляризацией , так что она склонна сводить к нулю веса слабых признаков. Логика интуитивно понятна: если функции явно бесполезны в простой модели, нет необходимости перетаскивать их в более сложную.

In [ ]:
# Synthetic example

from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline

x_data_generated, y_data_generated = make_classification()

rf = RandomForestClassifier(n_estimators=10, random_state=17)
pipe = make_pipeline(SelectFromModel(estimator=rf), logit)

print(cross_val_score(logit, x_data_generated, y_data_generated, 
                      scoring='neg_log_loss', cv=5).mean())
print(cross_val_score(rf, x_data_generated, y_data_generated, 
                      scoring='neg_log_loss', cv=5).mean())
print(cross_val_score(pipe, x_data_generated, y_data_generated, 
                      scoring='neg_log_loss', cv=5).mean())

Мы не должны забывать, что это не универсальный метод - поэтому это может сделать производительность хуже.

In [ ]:
#x_data, y_data = get_data() 
x_data = x_data_generated
y_data = y_data_generated

pipe1 = make_pipeline(StandardScaler(), 
                      SelectFromModel(estimator=rf), logit)

pipe2 = make_pipeline(StandardScaler(), logit)

print('LR + selection: ', cross_val_score(pipe1, x_data, y_data, 
                                          scoring='neg_log_loss', cv=5).mean())
print('LR: ', cross_val_score(pipe2, x_data, y_data, 
                              scoring='neg_log_loss', cv=5).mean())
print('RF: ', cross_val_score(rf, x_data, y_data, 
                              scoring='neg_log_loss', cv=5).mean())

### Перебор
Наконец, мы переходим к самому надежному методу, который также является наиболее сложным с точки зрения вычислений: тривиальный поиск по сетке (передор). Обучите модель на подмножестве признаков, сохраните результаты, повторите для разных подмножеств и сравните качество моделей, чтобы определить лучший набор признаков. Этот подход называется [исчерпывающий выбор признаков](http://rasbt.github.io/mlxtend/user_guide/feature_selection/ExhaustiveFeatureSelector/).

Поиск всех комбинаций обычно занимает слишком много времени, поэтому вы можете попытаться уменьшить пространство поиска. Зафиксируйте небольшое число N, перебирите все комбинации по N признаков, выберите лучшую комбинацию, а затем повторите комбинации из (N + 1) признаков так, чтобы предыдущая лучшая комбинация признаков была зафиксирована и рассматривался только один новый признак. Можно повторять до тех пор, пока мы не достигнем максимального количества характеристик или пока качество модели не перестанет значительно увеличиваться. Этот алгоритм называется [последовательный выбор признаков](http://rasbt.github.io/mlxtend/user_guide/feature_selection/SequentialFeatureSelector/).

Этот алгоритм можно перевернуть: начать с полного пространства объектов и удалять объекты один за другим, пока это не ухудшит качество модели или пока не будет достигнуто желаемое количество объектов.

In [ ]:
# Install mlxtend
from mlxtend.feature_selection import SequentialFeatureSelector

selector = SequentialFeatureSelector(logit, scoring='neg_log_loss', 
                                     verbose=2, k_features=3, forward=False, n_jobs=-1)

selector.fit(x_data, y_data)

Посмотрите, как этот подход был реализован в одном [простом, но элегантном ядре Kaggle](https://www.kaggle.com/arsenyinfo/easy-feature-selection-pipeline-0-55-at-lb).

Author: [Arseny Kravchenko](http://arseny.info/pages/about-me.html). Translated and edited by [Christina Butsko](https://www.linkedin.com/in/christinabutsko/), [Yury Kashnitskiy](https://yorko.github.io/), [Egor Polusmak](https://www.linkedin.com/in/egor-polusmak/), [Anastasia Manokhina](https://www.linkedin.com/in/anastasiamanokhina/), [Anna Larionova](https://www.linkedin.com/in/anna-larionova-74434689/), [Evgeny Sushko](https://www.linkedin.com/in/evgenysushko/) and [Yuanyuan Pao](https://www.linkedin.com/in/yuanyuanpao/). This material is subject to the terms and conditions of the [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/) license. Free use is permitted for any non-commercial purpose.